<a href="https://colab.research.google.com/github/luisfelipe100/Datos_Masivos/blob/main/Tarea1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalación de Spark

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession.builder.appName("Mi primera").getOrCreate()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,001 kB]
Ign:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
G

# Lectura de Datos

Los datos con los que se va a trabajar es un conjunto de imágenes. El conjunto consta de 94,126 imágenes de 100x100 píxeles de 141 clases distintas, entre frutas, vegetales y nueces. Se pretende entrenar un modelo de clasificación con estas imágenes.

## Clonación del Repositorio de GitHub

Se hace clonación del repositorio de GitHub que contiene las imágenes.
[Fruits-360](https://github.com/fruits-360/fruits-360-100x100)



In [ ]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.


In [ ]:
repository_url = "https://github.com/fruits-360/fruits-360-100x100"  # Replace with your repo URL
!git clone {repository_url}

Cloning into 'fruits-360-100x100'...
remote: Enumerating objects: 94555, done.
remote: Counting objects: 100% (3809/3809), done.
remote: Compressing objects: 100% (3782/3782), done.
remote: Total 94555 (delta 32), reused 3804 (delta 27), pack-reused 90746 (from 1)
Receiving objects: 100% (94555/94555), 723.85 MiB | 23.52 MiB/s, done.
Resolving deltas: 100% (33/33), done.
Updating files: 100% (94112/94112), done.


## Acceso a los Archivos con PySpark

Ahora usamos PySpark para leer los archivos dentro del repositorio clonado.

In [ ]:
import os
from pyspark.sql import SparkSession

# Assuming images are in the 'images' directory within 'fruits-360-100x100'
repo_name = "fruits-360-100x100"  # Replace with your repository folder name

# Fix: Use the repository name directly as the path
# os.path.abspath() is not needed here and was causing the issue
repo_path = "/content/fruits-360-100x100"

# Change to the repository directory
os.chdir(repo_path) #Change the current working directory

# Get list of image file paths
image_files = [os.path.join(root, f) for root, _, files in os.walk(repo_path) for f in files if f.endswith(('.png', '.jpg', '.jpeg'))]


# Create an RDD from the list
image_rdd = spark.sparkContext.parallelize(image_files)

# Example action: Count the number of images
print(f"Number of images: {image_rdd.count()}")

Number of images: 94110


In [ ]:
from PIL import Image

def load_image(file_path):
    try:
        img = Image.open(file_path)
        return file_path, img.size  # Return file path and image size as a tuple
    except:
        return file_path, None  # Return None if the file cannot be read

image_sizes_rdd = image_rdd.map(load_image)
image_sizes_rdd.take(5)  # Display the first 5 file paths and their corresponding sizes


[('/content/fruits-360-100x100/Test/Apple Crimson Snow 1/123_100.jpg',
  (100, 100)),
 ('/content/fruits-360-100x100/Test/Apple Crimson Snow 1/125_100.jpg',
  (100, 100)),
 ('/content/fruits-360-100x100/Test/Apple Crimson Snow 1/r_106_100.jpg',
  (100, 100)),
 ('/content/fruits-360-100x100/Test/Apple Crimson Snow 1/r_14_100.jpg',
  (100, 100)),
 ('/content/fruits-360-100x100/Test/Apple Crimson Snow 1/135_100.jpg',
  (100, 100))]

In [ ]:
def get_image_pixels(file_path):
    try:
        img = Image.open(file_path)
        return list(img.getdata())  # Return list of pixel values
    except:
        return []  # Return empty list if the image can't be opened

pixel_rdd = image_rdd.flatMap(get_image_pixels)
pixel_rdd.sample(False, 0.01).take(10)  # Display a sample of 10 RGB pixel values


[(255, 255, 253),
 (214, 197, 189),
 (167, 106, 88),
 (255, 255, 253),
 (255, 254, 255),
 (174, 101, 86),
 (250, 236, 235),
 (255, 253, 255),
 (254, 255, 253),
 (252, 255, 255)]

In [ ]:
[f for f in image_files if "Melon" in f][0:9]

['/content/fruits-360-100x100/Test/Melon Piel de Sapo 1/r2_277_100.jpg',
 '/content/fruits-360-100x100/Test/Melon Piel de Sapo 1/r2_260_100.jpg',
 '/content/fruits-360-100x100/Test/Melon Piel de Sapo 1/272_100.jpg',
 '/content/fruits-360-100x100/Test/Melon Piel de Sapo 1/r_0_100.jpg',
 '/content/fruits-360-100x100/Test/Melon Piel de Sapo 1/r2_254_100.jpg',
 '/content/fruits-360-100x100/Test/Melon Piel de Sapo 1/r_325_100.jpg',
 '/content/fruits-360-100x100/Test/Melon Piel de Sapo 1/287_100.jpg',
 '/content/fruits-360-100x100/Test/Melon Piel de Sapo 1/r_14_100.jpg',
 '/content/fruits-360-100x100/Test/Melon Piel de Sapo 1/r2_108_100.jpg']